In [ ]:
# 패키지 import
import datetime as dt
import pandas as pd
import requests
import time
import urllib.request #ㅇ
import re     
import konlpy

from bs4 import BeautifulSoup
from pandas import DataFrame
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [17]:
# 시각화  
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from PIL import Image             #워드클라우드용
from wordcloud import WordCloud   #워드클라우드용
fpath = "NotoSansCJKkr-Bold.otf"   #워드클라우드 국문지원을 위한 별도 폰트 path 설정
%matplotlib inline

# 기계 학습
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

ModuleNotFoundError: No module named 'PIL'

In [2]:
#크롬 드라이버 연결
delay=0.1
browser = Chrome()
browser.implicitly_wait(delay)

start_url  = 'https://www.youtube.com/channel/UCwx6n_4OcLgzAGdty0RWCoA/videos'
browser.get(start_url)  
browser.maximize_window()

body = browser.find_element_by_tag_name('body')#스크롤하기 위해 소스 추출
num_of_pagedowns = 30

#스크롤 다운
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.1)
    num_of_pagedowns -= 1

In [3]:
# 페이지 소스 받아오기
html0 = browser.page_source
html = BeautifulSoup(html0,'html.parser')

In [6]:
# 리스트 선언
title_list = [] #제목 리스트 생성
href_list = [] #주소 리스트 생성
viewcountmake_list = [] #조회수 크롤링 위한 리스트 생성
viewcount_list = [] #조회수 리스트 생성

In [7]:
# 구독자 수 저장
subsc = html.find(id="subscriber-count").text

# title 저장
for tr in html.find_all(id="video-title"):
    title = tr.get('title')
    title_list.append(title)

#href 태그 내용 저장
for tr in html.find_all(id="video-title"):
    href = tr.get('href')
    href_list.append(href)
    
# 조회수 저장
for tr in html.find_all('span', class_="style-scope ytd-grid-video-renderer"):
    viewcount = tr.get_text('span')
    viewcountmake_list.append(viewcount)
    
contents_count = len(title_list)

#구독자수, 동영상 수
print(subsc, ", 영상 개수:", contents_count)

구독자 353만명 , 영상 개수: 35


In [8]:
#전처리 전 title, 주소, 좋아요수 데이터프레임 생성
print(len(title_list), len(href_list), len(viewcountmake_list))
# viewcountmake_list의 경우 조회수와 등록일이 각각 생성되어 list크기가 다름 -> 데이터프레임 생성 불가

35 35 70


In [9]:
# 데이터 클리닝 작업(조회수와 등록일에서 등록일 제거)
for tr in range(0,len(viewcountmake_list),2):
    a = viewcountmake_list[tr]
    viewcount_list.append(a)
    
# 조회수 데이터 전처리
clean_viewcount = []
for i in viewcount_list:
    a = i[4:-2]
    amul = float(a)*10000
    clean_viewcount.append(int(amul))
print(clean_viewcount)

[3050000, 4810000, 5930000, 2290000, 5480000, 6480000, 6090000, 5120000, 7040000, 6650000, 7250000, 7010000, 2650000, 7580000, 6780000, 8780000, 7950000, 8310000, 10770000, 13860000, 9580000, 7320000, 1930000, 6300000, 8570000, 3490000, 5200000, 4920000, 6420000, 2850000, 5170000, 4440000, 4300000, 3590000, 4410000]


In [10]:
#title, 주소, 좋아요수 데이터프레임 생성
work_list = pd.DataFrame({'title':title_list, 'href':href_list, 'viewcount':clean_viewcount})
print(work_list)

                                                title                  href  \
0   역대급 카(car)오스💦 PD한테 바람맞고 카메라 차단 당해서 강제 탐사보도 돼버린...  /watch?v=VpkfjLPjz3s   
1   키즈카페가 그냥 커피라면 어린이집은 TOP야… 동심에 눈물폭발, 체력에 멘탈폭발, ...  /watch?v=Pw7w1_q3Q5M   
2   하루에 람보르기니 40대 본다구요?;; 외제차 받고 한류 스타까지 더블로 영접한 청...  /watch?v=ol9PmyoIq24   
3   길치가 배달알바하면 생기는 일.avi 🚫네비못봄, 경로이탈, 시간초과🚫 치킨 식혀서...  /watch?v=6NXzWfAHc_s   
4   (영상에서 🍺 냄새남) 워크맨 최초 NO시급 NO업무 NO사장(?) 낭만뿜뿜 놀멍쉬...  /watch?v=v5MYPb_T4Co   
5   잠시 쉬다오겠습니다... 포상휴가인 줄 알고 떠났다가 저세상 서비스하고 온 5성급 ...  /watch?v=W2Kpi5zDABE   
6   이정재, 정웅인 대한민국 명배우 총집합⭐ 다 된 드라마에 장성규 NG 뿌리기...?...  /watch?v=l502xg11uNM   
7       (궁서체) JOB것들을 위해... 이번만큼은 진지합니다.... ㅣ워크맨 X 사람인  /watch?v=byjgWjqoCvw   
8   *혈압주의* 옷가게 알바하다가 최악의 변태진상(손놈) 😈만난 썰;; 상상초월 의류매...  /watch?v=jEA3p1ekzSw   
9   🎉 300만 돌파 축하, 게드립 생선드립 다 모아봄🎉 해산물 제철 기념 날로 먹는 ...  /watch?v=bvN7YelS34A   
10  민속촌 보내놨더니 여기서도 아마존;; 구역 안 가리고 저세상드립 폭발해버린 민속촌 ...  /watch?v=aqpqoNk7GTs   
11  ⚠안전제일 ⚠ Job것들이 제일로 원했던 장성규의 고생길♥ 일하다가 5줌 쌀 뻔한 ..

In [11]:
work_list.head()

,title,href,viewcount
0,역대급 카(car)오스💦 PD한테 바람맞고 카메라 차단 당해서 강제 탐사보도 돼버린...,/watch?v=VpkfjLPjz3s,3050000
1,"키즈카페가 그냥 커피라면 어린이집은 TOP야… 동심에 눈물폭발, 체력에 멘탈폭발, ...",/watch?v=Pw7w1_q3Q5M,4810000
2,하루에 람보르기니 40대 본다구요?;; 외제차 받고 한류 스타까지 더블로 영접한 청...,/watch?v=ol9PmyoIq24,5930000
3,"길치가 배달알바하면 생기는 일.avi 🚫네비못봄, 경로이탈, 시간초과🚫 치킨 식혀서...",/watch?v=6NXzWfAHc_s,2290000
4,(영상에서 🍺 냄새남) 워크맨 최초 NO시급 NO업무 NO사장(?) 낭만뿜뿜 놀멍쉬...,/watch?v=v5MYPb_T4Co,5480000


In [12]:
work_list.sort_values(by=['viewcount'], axis=0, ascending=False)

,title,href,viewcount
19,🎉구독자 100만 기념 역대급 레전드🎉 인싸계 최종보스몹 에버랜드 알바 리뷰 1편 ...,/watch?v=UQYCWLrCUy4,13860000
18,※기내 💊류 반입 금지※ 약 빤 장성규의 꿈의 직장★ 상큼터지는 항공사 직업 리뷰ㅣ...,/watch?v=m8JrVquHNsY,10770000
20,사장님도 모르는 피자 토핑 꿀조합... 🍕?! 루팡 클라스가 다른 천조국 피자집 알...,/watch?v=9OUaRBazbaQ,9580000
15,🎉200만 구독자 특집이 되어버린 에버랜드 2탄!! 🎉 역대급 초과근무에 파업 선언...,/watch?v=tQjfEFg1DR8,8780000
24,역대급 가’족’같은 분위기🏠 머리카락 대신 목 날아갈 뻔한 미용실 알바 리뷰ㅣ워크맨...,/watch?v=hSdwuOj2ecA,8570000
17,🔥구독자 요청 폭주🔥편돌이 시켰더니 먹방만 남아버린 국민 알바 편의점 알바 리뷰 ㅣ...,/watch?v=8NH2FBXUOt4,8310000
16,장성규 카트라이더 🌈무지개장갑썰 ㄹㅇ이었네? 놀면서 돈벌기 끝판王 게임회사 리뷰ㅣ워...,/watch?v=ufV7PIokoT0,7950000
13,비오면 X을 싼다고요...? 역대급 정신없음류 甲 난이도⭐️X100 대환장 술집 알...,/watch?v=KVGK856HUjE,7580000
21,JTBC로 돌아온 도비 장성규?! 오디오 빌 틈 1도 없는 녹즙 배달 인생 알바 리...,/watch?v=vZ-usD5ufSc,7320000
10,민속촌 보내놨더니 여기서도 아마존;; 구역 안 가리고 저세상드립 폭발해버린 민속촌 ...,/watch?v=aqpqoNk7GTs,7250000


In [69]:
# 영상 본문 key(제목, 조회수, 좋아요, 싫어요, 본문내용, 전체 댓글 수) 추출 - 1
#################### 시간 소요 많음

title_num = 0;
n_title_list =[] # 영상별 제목 리스트 생성
story_sub_list = [] # 영상 본문내용 리스트
n_viewcount_list = [] #영상별 조회수 리스트 생성
n_like_list = [] #영상별 좋아요 리스트 생성
n_unlike_list = [] #영상별 싫어요 리스트 생성
n_date_list = [] #영상별 작성일 리스트 생성
n_href_list = [] # 영상별 주소 리스트 생성
start_url  = 'https://www.youtube.com/'

# 데이터
for i in href_list: # :2 뺴면 전체 데이터, 시간 단축을 위해 2개 샘플만 시행
    url = start_url + i
    browser.get(url)
    print(i)  # 추출 주소값
    n_href_list.append(i)
    
    source = browser.page_source
    bs = BeautifulSoup(source, 'html.parser')
    bsstory = BeautifulSoup(source, "html.parser")
    
    
    # 영상에 대한 기본 정보 수집
    info1 = bs.find('div',{'id':'info-contents'})
    
    #n_title = "" #초기화 계속
    n_title = info1.find('h1',{'class':'title style-scope ytd-video-primary-info-renderer'}).text 
    n_title_list.append(n_title)
    n_viewcount =info1.find('yt-view-count-renderer',{'class':'style-scope ytd-video-primary-info-renderer'}).find_all('span')[0].text  
    n_viewcount_list.append(n_viewcount)
    n_like = info1.find('div',{'id':'top-level-buttons'}).find_all('yt-formatted-string')[0].text #좋아요수
    n_like_list.append(n_like)
    n_unlike = info1.find('div',{'id':'top-level-buttons'}).find_all('yt-formatted-string')[1].text 
    n_unlike_list.append(n_unlike)

    # 영상 본문 내용 추출
    story_sub = bsstory.find('yt-formatted-string', class_='content style-scope ytd-video-secondary-info-renderer').text
    story_sub_list.append(story_sub)
    title_num = title_num + 1; 
    #print(title_num, "번째 영상 본문 추출 끝")
    
# 리스트에 모두 입력되었는지 확인
print("제목: ", n_title_list)
print("조회수: ", n_viewcount_list)
print("좋아요: ", n_like_list)
print("싫어요: ",n_unlike_list)


# 조회수 데이터 전처리
clean_n_viewcount = []
for i in n_viewcount_list:
    a = i[4:-1]
    clean_n_viewcount.append(a)
print(clean_n_viewcount)

# 좋아요 데이터 전처리
clean_n_like_list = []
for i in n_like_list:
    if i[-1] == '만':
        a = i[:-1]
        amul = float(a)*10000
        clean_n_like_list.append(int(amul))
    else:
        clean_n_like_list.append(i)
print(clean_n_like_list)

# 싫어요 데이터 전처리
clean_n_unlike_list = []
for i in n_unlike_list:
    if i[-1] == '천':
        a = i[:-1]
        amul = float(a)*1000
        clean_n_unlike_list.append(int(amul))
    else:
        clean_n_unlike_list.append(i)
print(clean_n_unlike_list)

#dataframe 생성
n_work_list = pd.DataFrame({'제목':n_title_list, '주소':n_href_list, '조회수':clean_n_viewcount,
                           '좋아요':clean_n_like_list, '싫어요':clean_n_unlike_list, '본문내용':story_sub_list})

/watch?v=VpkfjLPjz3s
/watch?v=Pw7w1_q3Q5M
/watch?v=ol9PmyoIq24
/watch?v=6NXzWfAHc_s
/watch?v=v5MYPb_T4Co
/watch?v=W2Kpi5zDABE
/watch?v=l502xg11uNM
/watch?v=byjgWjqoCvw
/watch?v=jEA3p1ekzSw
/watch?v=bvN7YelS34A
/watch?v=aqpqoNk7GTs
/watch?v=RjGbCptFlFM
/watch?v=JX3X5W8HylE
/watch?v=KVGK856HUjE
/watch?v=k5YaBcEBtGQ
/watch?v=tQjfEFg1DR8
/watch?v=ufV7PIokoT0
/watch?v=8NH2FBXUOt4
/watch?v=m8JrVquHNsY
/watch?v=UQYCWLrCUy4
/watch?v=9OUaRBazbaQ
/watch?v=vZ-usD5ufSc
/watch?v=9KfKfYHr1HY
/watch?v=I8ohd4o0cvI
/watch?v=hSdwuOj2ecA
/watch?v=LESegodaX-c
/watch?v=zeY3P0kPa0I
/watch?v=gScLohLMiCA
/watch?v=ht0Zl9BJCwQ
/watch?v=SKsKdLNRDDk
/watch?v=A0as_-0ObgU
/watch?v=QKTEbrFDbEc
/watch?v=MnJqDkNwIsk
/watch?v=Das7SCjARAk
/watch?v=DodhBgfzY_c
제목:  ['역대급 카(car)오스💦 PD한테 바람맞고 카메라 차단 당해서 강제 탐사보도 돼버린 타이어 매장 알바 리뷰 (feat. 민속촌 패밀리)ㅣ워크맨 ep.31', '키즈카페가 그냥 커피라면 어린이집은 TOP야… 동심에 눈물폭발, 체력에 멘탈폭발, 오디오감독님 고막폭발 💀 어린이집 교사 리뷰ㅣ워크맨 ep.30', '하루에 람보르기니 40대 본다구요?;; 외제차 받고 한류 스타까지 더블로 영접한 청담동 주유소 알바 리뷰ㅣ워크맨 ep.29', '길치가 배달알바하면 생기

In [70]:
n_work_list.head()

,제목,주소,조회수,좋아요,싫어요,본문내용
0,역대급 카(car)오스💦 PD한테 바람맞고 카메라 차단 당해서 강제 탐사보도 돼버린...,/watch?v=VpkfjLPjz3s,"3,090,770",52000,1000,"JOB것들아~~~♥ 인력소장이다,,,,~~~😎😎😎\n운전하는 잡것덜~~~ 차는,,,..."
1,"키즈카페가 그냥 커피라면 어린이집은 TOP야… 동심에 눈물폭발, 체력에 멘탈폭발, ...",/watch?v=Pw7w1_q3Q5M,"4,815,437",77000,1300,"JOB것들아~~~♥ 인력소장이다,,,,~~~😎 😎 😎\n진짜,,,,, 초등학교든,,..."
2,하루에 람보르기니 40대 본다구요?;; 외제차 받고 한류 스타까지 더블로 영접한 청...,/watch?v=ol9PmyoIq24,"5,939,194",100000,1400,"JOB것들아~~~♥ 인력소장이다,,,,~~~😎 😎 😎\r\n주유소,,알바도~~항상 ..."
3,"길치가 배달알바하면 생기는 일.avi 🚫네비못봄, 경로이탈, 시간초과🚫 치킨 식혀서...",/watch?v=6NXzWfAHc_s,"2,298,039",28000,775,"JOB것들아~~~♥ 인력소장이다,,,,~~~😎 😎 😎\n요즘,,,갑자기 화아아악,,..."
4,(영상에서 🍺 냄새남) 워크맨 최초 NO시급 NO업무 NO사장(?) 낭만뿜뿜 놀멍쉬...,/watch?v=v5MYPb_T4Co,"5,486,912",90000,1400,"JOB것들아~~~♥ 인력소장이다,,,,~~~😎 😎 😎\n히야아 요즘 내 동년배덜,,..."


In [45]:
# 영상별 댓글 추출
title_num = 0;
youtube_pd = pd.DataFrame() # 영상 당 댓글 df 생성용
total_youtube_pd = pd.DataFrame() #최종 df 생성용
str_youtube_comments_len = [] # 영상별 댓글수 리스트 생성
 # 좋아요수 클리닝을 위한 리스트 생성

start_url  = 'https://www.youtube.com/'
for i in href_list: # :2 뺴면 전체 데이터, 시간 단축을 위해 2개 샘플만 시행
    url = start_url + i
    browser.get(url)  
    print(i)  # 추출 주소값
    source = browser.page_source
    bs = BeautifulSoup(source, "html.parser")

    ############################
    # 댓글 추출 코드
    reply_list = [] #댓글 리스트 생성
    body = browser.find_element_by_tag_name('body')#스크롤하기 위해 소스 추출
    num_of_pagedowns = 10

    #스크롤 다운
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        num_of_pagedowns -= 1
    
        html0 = browser.page_source
        html = BeautifulSoup(html0,'html.parser')

        youtube_user_IDs = html.select('div#header-author > a > span')
        youtube_comments = html.select('yt-formatted-string#content-text')
        youtube_likes = html.select('div#toolbar > span')
    
        str_youtube_userIDs = []
        str_youtube_comments = []
        str_youtube_likes = []
        str_youtube_title_number = [] #영상 타이틀 넘버
             
           
    for i in range(len(youtube_user_IDs)):
        str_tmp = str(youtube_user_IDs[i].text)
        str_tmp = str_tmp.replace('\n', '')
        str_tmp = str_tmp.replace('\t', '')
        str_tmp = str_tmp.replace('               ','')
        str_youtube_userIDs.append(str_tmp)

        str_tmp = str(youtube_comments[i].text)
        str_tmp = str_tmp.replace('\n', '')
        str_tmp = str_tmp.replace('\t', '')
        str_tmp = str_tmp.replace('               ', '')
        str_youtube_comments.append(str_tmp)
    
        str_tmp = str(youtube_likes[i].text)
        str_tmp = str_tmp.replace('\n', '')
        str_tmp = str_tmp.replace('\t', '')
        str_tmp = str_tmp.replace('               ','')
        str_youtube_likes.append(str_tmp)
        
        # 어떤 영상인지 넘버링으로 확인
        str_youtube_title_number.append(title_num)
        #print("title_num: ", title_num) #테스트
        
        clean_str_youtube_likes = []
        # 좋아요수 전처리, 좋아요수가 str+공백으로 들어가 있어 공백 제거 후 계산
        for i in str_youtube_likes:
            i = i.replace(" ", '')
            if i.find('만') !=-1:
                a = i.replace('만', '')
                amul = float(a)*10000
                clean_str_youtube_likes.append(int(amul))
            elif i.find('천') != -1:
                a = i.replace('천', '')
                amul = float(a)*1000
                clean_str_youtube_likes.append(int(amul))
            else:
                clean_str_youtube_likes.append(int(i))
        
         # 리뷰 & 좋아요수 데이터 프레임 생성
        youtube_pd = pd.DataFrame({"ID":str_youtube_userIDs, "Comment":str_youtube_comments, 
                                   "Like":clean_str_youtube_likes, "Title_number":str_youtube_title_number})
    total_youtube_pd = pd.concat([total_youtube_pd, youtube_pd]) 
    print(len(youtube_pd))
    
    str_youtube_comments_len.append(len(youtube_pd))
        
    print("")
   
   # 본문 및 댓글 취합내용 확인_부하로 인해 상위 1, 2개만 추출


print("* 본문 추출 개수: ", len(story_sub_list))   #추출한 본문 개수 확인
print("* 댓글 추출 개수: ", len(total_youtube_pd)) #추출한 댓글 개수 확인

/watch?v=VpkfjLPjz3s
20

/watch?v=Pw7w1_q3Q5M
20

/watch?v=ol9PmyoIq24
20

/watch?v=6NXzWfAHc_s
20

/watch?v=v5MYPb_T4Co
20

/watch?v=W2Kpi5zDABE
20

/watch?v=l502xg11uNM
20

/watch?v=byjgWjqoCvw
20

/watch?v=jEA3p1ekzSw
20

/watch?v=bvN7YelS34A
20

/watch?v=aqpqoNk7GTs
20

/watch?v=RjGbCptFlFM
20

/watch?v=JX3X5W8HylE
20

/watch?v=KVGK856HUjE
20

/watch?v=k5YaBcEBtGQ
20

/watch?v=tQjfEFg1DR8
20

/watch?v=ufV7PIokoT0
20

/watch?v=8NH2FBXUOt4
20

/watch?v=m8JrVquHNsY
20

/watch?v=UQYCWLrCUy4
20

/watch?v=9OUaRBazbaQ
20

/watch?v=vZ-usD5ufSc
20

/watch?v=9KfKfYHr1HY
20

/watch?v=I8ohd4o0cvI
20

/watch?v=hSdwuOj2ecA
20

/watch?v=LESegodaX-c
20

/watch?v=zeY3P0kPa0I
20

/watch?v=gScLohLMiCA
20

/watch?v=ht0Zl9BJCwQ
20

/watch?v=SKsKdLNRDDk
20

/watch?v=A0as_-0ObgU
20

/watch?v=QKTEbrFDbEc
20

/watch?v=MnJqDkNwIsk
20

/watch?v=Das7SCjARAk
20

/watch?v=DodhBgfzY_c
20

* 본문 추출 개수:  35
* 댓글 추출 개수:  700


In [48]:
print(str_youtube_comments_len)

[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]


In [49]:
#추출한 본문 예시 1개 확인
print("예시 1개", print(story_sub_list[0]))

JOB것들아~~~♥ 인력소장이다,,,,~~~😎😎😎
운전하는 잡것덜~~~ 차는,,, 타이어가 생명인거,,, 알제~~~ㅎ
펑크나면,,,~ 후딱후딱 바꾸러가고~~ 미루다가 큰일난다~~~!! 

세상의 모든 job것들을리뷰한다 [워크맨]
▶ 매주 금 오후 6시공개

#워크맨 #workman #장성규 #타이어 #넥센 #자동차
#직업리뷰 #알바리뷰 #장성규 #스튜디오룰루랄라
예시 1개 None


In [50]:
# 본문내용 데이터 전처리
clean_story_sub_list = []
for i in range(len(story_sub_list)):
        str_tmp = str(story_sub_list[i])
        # print(str_tmp)
        str_tmp = str_tmp.replace('#', '')
        str_tmp = str_tmp.replace('\n', '')
        str_tmp = str_tmp.replace('\r', '')
        str_tmp = str_tmp.replace('세상의 모든 job것들을 리뷰한다 [워크맨]', '')
        str_tmp = str_tmp.replace('▶ 매주 금 오후 6시 공개','')
        str_tmp = str_tmp.replace('▶ 구독/좋아요/알람 설정','')
        str_tmp = str_tmp.replace('세상의 모든 job것들을리뷰한다 [워크맨]▶ 매주 금 오후 6시공개','')
        clean_story_sub_list.append(str_tmp)

print(clean_story_sub_list)

['JOB것들아~~~♥ 인력소장이다,,,,~~~😎😎😎운전하는 잡것덜~~~ 차는,,, 타이어가 생명인거,,, 알제~~~ㅎ펑크나면,,,~ 후딱후딱 바꾸러가고~~ 미루다가 큰일난다~~~!! 워크맨 workman 장성규 타이어 넥센 자동차직업리뷰 알바리뷰 장성규 스튜디오룰루랄라', 'JOB것들아~~~♥ 인력소장이다,,,,~~~😎 😎 😎진짜,,,,, 초등학교든,,,~중학교든~~ 어린이집이든!!!!선생님들 너~,,,무 대단하신거여,,, ,다들 리스펙~~~~~ 워크맨 workman 장성규 어린이집 어린이집교사직업리뷰 알바리뷰 장성규 스튜디오룰루랄라', 'JOB것들아~~~♥ 인력소장이다,,,,~~~😎 😎 😎주유소,,알바도~~항상 안전이 젤 중요한거시다~~~,,,,알긋지~!?항상 불조심하고~~~차조심하고,,,,!!! 워크맨 workman 장성규 주유소알바 GS칼텍스직업리뷰 알바리뷰 장성규 스튜디오룰루랄라 엑소 EXO', 'JOB것들아~~~♥ 인력소장이다,,,,~~~😎 😎 😎요즘,,,갑자기 화아아악,,, 추워져갖꼬~~~!! 집에서덜,, 배달 많이 시켜묵는디,,,배달하는 알바생덜,,,, 극한 직업이니께,,, 감사하다고~,,, 인사 한마디씩은 해주자고오~ 워크맨 workman 장성규 배달알바 배민 배달음식직업리뷰 알바리뷰 장성규 스튜디오룰루랄라', 'JOB것들아~~~♥ 인력소장이다,,,,~~~😎 😎 😎히야아 요즘 내 동년배덜,,, 다 제주도 가믄 게스트하우스 간다더라고,,,,~~?울 잡것들도,,,~ 제주도서 게하체험해보구,~,,, 술은 너무 마니 마시지말고덜~~~!!!ㅎ 워크맨 workman 장성규 게하스탭 제주도직업리뷰 알바리뷰 장성규 스튜디오룰루랄라', 'JOB것들아~~~♥ 인력소장이다,,,,~~~😎 😎 😎장성규 스튜핏이 잘 어울리길래,,, 호텔리어 옷도 찰떡아니냐~~~부내나는 알바,,,라 꿀~~일줄 알았는디~~~~ㅎ남들 쉴 때 못쉬는 힘든일이여,,,,ㅜ 워크맨 workman 장성규 호텔리어 제주도직업리뷰 알바리뷰 장성규 스튜디오룰루랄라', 'JOB것들아~~~ 인력소장이다,,,

In [72]:
# title, 주소, 조회수, 본문내용 추가한 데이터 프레임
work_list_update = pd.DataFrame({'제목':n_title_list, '주소':n_href_list, '조회수':clean_n_viewcount,
                           '좋아요':clean_n_like_list, '싫어요':clean_n_unlike_list, '댓글수':str_youtube_comments_len, '본문내용':clean_story_sub_list})

work_list_update.head()

,제목,주소,조회수,좋아요,싫어요,댓글수,본문내용
0,역대급 카(car)오스💦 PD한테 바람맞고 카메라 차단 당해서 강제 탐사보도 돼버린...,/watch?v=VpkfjLPjz3s,"3,090,770",52000,1000,20,"JOB것들아~~~♥ 인력소장이다,,,,~~~😎😎😎운전하는 잡것덜~~~ 차는,,, 타..."
1,"키즈카페가 그냥 커피라면 어린이집은 TOP야… 동심에 눈물폭발, 체력에 멘탈폭발, ...",/watch?v=Pw7w1_q3Q5M,"4,815,437",77000,1300,20,"JOB것들아~~~♥ 인력소장이다,,,,~~~😎 😎 😎진짜,,,,, 초등학교든,,,~..."
2,하루에 람보르기니 40대 본다구요?;; 외제차 받고 한류 스타까지 더블로 영접한 청...,/watch?v=ol9PmyoIq24,"5,939,194",100000,1400,20,"JOB것들아~~~♥ 인력소장이다,,,,~~~😎 😎 😎주유소,,알바도~~항상 안전이 ..."
3,"길치가 배달알바하면 생기는 일.avi 🚫네비못봄, 경로이탈, 시간초과🚫 치킨 식혀서...",/watch?v=6NXzWfAHc_s,"2,298,039",28000,775,20,"JOB것들아~~~♥ 인력소장이다,,,,~~~😎 😎 😎요즘,,,갑자기 화아아악,,, ..."
4,(영상에서 🍺 냄새남) 워크맨 최초 NO시급 NO업무 NO사장(?) 낭만뿜뿜 놀멍쉬...,/watch?v=v5MYPb_T4Co,"5,486,912",90000,1400,20,"JOB것들아~~~♥ 인력소장이다,,,,~~~😎 😎 😎히야아 요즘 내 동년배덜,,, ..."


In [52]:
# 0번째 영상에서 추출한 댓글 DataFrame
youtube_pd.head()

,ID,Comment,Like,Title_number
0,워크맨-Workman,🐜🐜워크맨 인력소🐜🐜\r워크맨의 리뷰를 원하는 job것이 있다면↓댓글 추천↓\r\r...,1900,0
1,HS Kim,여직원 왜? 일 잘하는 거 같은데... 예의없는것도 아니고 워터파크는 삐끗하면 사고...,1900,0
2,김준,워터파크 크기와 수준에 비해 청소용품 관련이 너무 열악한거 같다. 놀램. 저 정도의...,13000,0
3,후쿠티비 쇼핑몰놈들,담당 선배여자분 워크맨 시리즈중 프로페셔널한편에 속하는거 같은데....,13000,0
4,Gray oasis,아니ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ군대...,7400,0


In [53]:
# 전체 영상에서 추출한 댓글 DataFrame
total_youtube_pd[:100]

,ID,Comment,Like,Title_number
0,워크맨-Workman,"JOB것들아~~~♥ 인력소장이다,,,,~~~😎😎😎이번에는,,,특이한,,,~ 알바 시...",7100,0
1,잡코리아USA - 알면 쉬운 미국 취업 TV,민속촌 갑분툭 일하는거 겁나 웃기네 ㅋㅋㅋㅋㅋ 끝난거지 는 또 뭐야 ㅋㅋㅋㅋㅋㅋㅋ...,7100,0
2,김성희,ㅋㅋㅌㅋㅋㅋㅋㅋㅋㅋ아니 민속촌 알바하시는 분들은 왜 친절하게 도와주시는 건데ㅋㅋㅌㅋ...,3100,0
3,탱귤 탱귤,구독자가 왜 장성규를 모르는거얔ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...,3100,0
4,주네JUNE,아진짴ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 갈수록 자막도 고퀄이 되어가는 것 같아요 너무 웃겨 ㅠㅠㅠ...,1800,0
...,...,...,...,...
15,mabel,this is such an interesting job! never saw any...,761,0
16,김봉수,휘닉스평창 제설팀입니다. 제설팀도 일일체험 와주셨으면 재밌을거 같습니다.다른방송사에...,1200,0
17,최현규,저 사장님 마인드가 너무 좋다.....여행자에 대한 존중이랑 그 철학이 예쁘다고 해야하나,1200,0
18,이진규,칫솔이 왜 응원봉이 돼 ㅋㅋㅋㅋㅋㅋㅋㅋ,47,0


In [73]:
# 워크맨 동영상 csv 저장
work_list_update.to_csv("work.csv", mode='w', encoding="utf-8-sig")

# 워크맨 댓글 csv 저장
total_youtube_pd.to_csv("work_reply.csv", mode='w', encoding="utf-8-sig")